# Imports

In [2]:
import os
import numpy as np
import face_recognition
import cv2

# Taking images

In [4]:
name = input("Please enter user name: ")

os.mkdir("images/" + name)

cam = cv2.VideoCapture(0)

cv2.namedWindow("test")

img_counter = 0

while True:
    ret, frame = cam.read()
    if not ret:
        print("failed to grab frame")
        break
    cv2.imshow("test", frame)

    k = cv2.waitKey(1)
    if k%256 == 27:
        # ESC pressed
        print("Escape hit, closing...")
        break
    elif k%256 == 32:
        # SPACE pressed
        img_name = "images/{}/opencv_frame_{}.png".format(name, img_counter)
        cv2.imwrite(img_name, frame)
        print("{} written!".format(img_name))
        img_counter += 1

cam.release()

cv2.destroyAllWindows()

images/Maher/opencv_frame_0.png written!
images/Maher/opencv_frame_1.png written!
images/Maher/opencv_frame_2.png written!
Escape hit, closing...


# Face Detection

In [5]:
class FaceDetectorReady:
    def __init__(self):
        self.frame_resizing = 1

    def detect_borders(self, img):
        resized_img = cv2.resize(img, (0, 0), fx=self.frame_resizing, fy=self.frame_resizing)
        rgb_img = cv2.cvtColor(resized_img, cv2.COLOR_BGR2RGB)
        face_locations = face_recognition.face_locations(rgb_img)
        final_imgs = []
        for face_loc in face_locations:
            # top, right, bottom, left = face_loc[0], face_loc[1], face_loc[2], face_loc[3]
            y1, x2, y2, x1 = face_loc[0], face_loc[1], face_loc[2], face_loc[3]
            final_imgs.append(resized_img[y1:y2, x1:x2])

        try:
            return final_imgs, face_locations
        except:
            return None, None

# LBPH

In [6]:

class LBPHfromScratch:
    def __init__(self):
        self.R = 1
        self.P = 8
        self.filter_size = 3
        # Anti-clockwise (right -> up + right -> up -> up + left -> left -> down + left -> down -> down + right)
        self.filter_lbp = np.array([[2, 1], [2, 0], [1, 0], [0, 0], [0, 1], [0, 2], [1, 2], [2, 2]])

    def Compute_LBP(self, img):
        # Determine the dimensions of the input image.
        height = img.shape[0]
        width = img.shape[1]

        if width < self.filter_size or height < self.filter_size:
            print("Size not correct!")
            return

        out_width = width - self.filter_size + 1
        out_height = height - self.filter_size + 1

        reference_matrix = img[1:1 + out_height, 1:1 + out_width]

        out_img = np.zeros((out_height, out_width))

        for i in range(0, 8):
            step_x, step_y = self.filter_lbp[i]

            sliding_matrix = img[step_y:step_y + out_height, step_x:step_x + out_width]

            flags = np.greater_equal(sliding_matrix, reference_matrix)

            exponent = np.power(2, i)
            out_img = out_img + (flags * exponent)

        return out_img.astype(np.uint8)


# Histogram Comparison

In [7]:
class Matcher:
    def __init__(self):
        self.n_bins = 256

    def match(self, refs, lbp):
        best_score = float('inf')
        best_name = None
        hist = cv2.calcHist([lbp], [0], None, [256], [0, 256])
        hist /= hist.sum()

        for name, ref_hist in refs:
            score = cv2.compareHist(hist, ref_hist, cv2.HISTCMP_CHISQR)
            if score < best_score:
                best_score = score
                best_name = name
        best_score = best_score * 100
        return best_name, best_score

# Training

In [8]:
face_detect = FaceDetectorReady()
lbph_2 = LBPHfromScratch()
classifier = Matcher()

In [10]:
enc_list = []
img_list = []

path = 'images'

directory_contents = os.listdir(path)
for directory in directory_contents:
    image_names = os.listdir(path + "/" + directory)
    for item in image_names:
        img_name = directory
        img_curr = cv2.imread(path + "/" + directory + "/" + item)
        cropped_img, face_loc_img = face_detect.detect_borders(img_curr)
        if cropped_img is None:
            continue
        for cropped_img_item in cropped_img:
            gray_img = cv2.cvtColor(cropped_img_item, cv2.COLOR_RGB2GRAY)
            temp_img = lbph_2.Compute_LBP(gray_img)
            ref_hist = cv2.calcHist([temp_img], [0], None, [256], [0, 256])
            ref_hist /= ref_hist.sum()

            # Saving the image
            cv2.imwrite("mine/" + img_name + ".jpg", temp_img)
            enc_list.append((img_name, ref_hist))

weights_array = np.array(enc_list, dtype=object)

with open('weights.npy', 'wb') as f:
    np.save(f, weights_array)

# Test

In [11]:
cap = cv2.VideoCapture(0)
with open('weights.npy', 'rb') as f:
    weights_array = np.load(f, allow_pickle=True)
while True:
    ret, frame = cap.read()

    # Detect Faces
    try:
        cropped_test_img, face_loc_test_img = face_detect.detect_borders(frame)

        for cropped_img_item, face_loc in zip(cropped_test_img, face_loc_test_img):
            gray_test_img = cv2.cvtColor(cropped_img_item, cv2.COLOR_RGB2GRAY)
            lbp_img = lbph_2.Compute_LBP(gray_test_img)
            n, s = classifier.match(weights_array, lbp_img)

            y1, x2, y2, x1 = face_loc[0], face_loc[1], face_loc[2], face_loc[3]

            cv2.putText(frame, n + " " + str(s), (x1, y1 - 10), cv2.FONT_HERSHEY_DUPLEX, 1, (0, 0, 200), 2)
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 200), 4)
    except Exception as e:
        print(e)

    cv2.imshow("User", frame)

    key = cv2.waitKey(1)
    if key == 27:
        break

cap.release()
cv2.destroyAllWindows()

cv2.waitKey(0)


-1